In [3]:
import torch

def load_file(path):
  with open(path) as f:
    return f.read().strip().splitlines()

In [4]:
def parse_line(line):
  direction = line[0]
  distance = int(line[1:])
  return dict(direction=direction, distance=distance)

def parse_file(content):
  return [parse_line(line) for line in content] 

In [10]:
instructions = parse_file(load_file('example.txt'))

In [101]:
def get_initial_state():
  grid_size = 1_000
  start = grid_size // 2
  head = torch.tensor([start, start], dtype=torch.float32)
  tail = torch.tensor([start, start], dtype=torch.float32)
  visits = torch.zeros((grid_size, grid_size), dtype=torch.float32)

  return [head, tail, visits]
  

In [17]:
[a, b] = get_initial_state()

In [38]:
[a,b] = get_initial_state()
a[0] += 1
a[1] += 2
print(a.dist(b))

tensor(2.2361)


In [102]:
def run_instruction(state, instruction):
  [head, tail, visits] = state
  direction = instruction['direction']
  distance = instruction['distance']
  for i in range(distance):
    if direction == 'U':
      head[1] += 1
    elif direction == 'D':
      head[1] -= 1
    elif direction == 'L':
      head[0] -= 1
    elif direction == 'R':
      head[0] += 1
    else:
      raise Exception('Unknown direction')
      
    if (head.dist(tail) >= 2):
      possible_moves = torch.tensor([[-1, 0], [1, 0], [0, -1], [0, 1], [1, 1], [-1, -1], [1, -1], [-1, 1]]) + tail
      distances = torch.cdist(head.unsqueeze(0), possible_moves)
      tail = possible_moves[torch.argmin(distances)]
    print(tail)
    visits[int(tail[0]), int(tail[1])] += 1
    
  return [head, tail, visits]
    

In [103]:
def run_instructions(instructions):
  state = get_initial_state()
  for instruction in instructions:
    state = run_instruction(state, instruction)
  return state

In [104]:
(run_instructions(instructions)[2] > 0).sum()

tensor([500., 500.])
tensor([501., 500.])
tensor([502., 500.])
tensor([503., 500.])
tensor([503., 500.])
tensor([504., 501.])
tensor([504., 502.])
tensor([504., 503.])
tensor([504., 503.])
tensor([503., 504.])
tensor([502., 504.])
tensor([502., 504.])
tensor([502., 504.])
tensor([502., 504.])
tensor([503., 503.])
tensor([504., 503.])
tensor([504., 503.])
tensor([504., 503.])
tensor([504., 503.])
tensor([503., 502.])
tensor([502., 502.])
tensor([501., 502.])
tensor([501., 502.])
tensor([501., 502.])


tensor(13)

In [105]:
(run_instructions(parse_file(load_file('input.txt')))[2] > 0).sum()

tensor([500., 500.])
tensor([500., 500.])
tensor([500., 500.])
tensor([500., 500.])
tensor([500., 500.])
tensor([500., 500.])
tensor([500., 500.])
tensor([500., 500.])
tensor([499., 499.])
tensor([498., 499.])
tensor([498., 499.])
tensor([497., 500.])
tensor([497., 500.])
tensor([496., 501.])
tensor([496., 501.])
tensor([495., 500.])
tensor([495., 500.])
tensor([494., 499.])
tensor([494., 499.])
tensor([493., 498.])
tensor([493., 498.])
tensor([493., 498.])
tensor([493., 497.])
tensor([493., 497.])
tensor([493., 497.])
tensor([493., 497.])
tensor([493., 497.])
tensor([493., 497.])
tensor([493., 497.])
tensor([493., 497.])
tensor([494., 496.])
tensor([495., 496.])
tensor([495., 496.])
tensor([495., 496.])
tensor([495., 496.])
tensor([495., 496.])
tensor([495., 496.])
tensor([496., 495.])
tensor([496., 495.])
tensor([495., 494.])
tensor([495., 494.])
tensor([494., 493.])
tensor([494., 493.])
tensor([494., 493.])
tensor([494., 493.])
tensor([493., 492.])
tensor([493., 491.])
tensor([493.,

tensor(6563)

In [147]:
def get_initial_state(rope_length=10):
  grid_size = 1_000
  start = grid_size // 2
  rope = torch.tensor([start, start] * rope_length , dtype=torch.float32).view(rope_length, 2)
  visits = torch.zeros((grid_size, grid_size), dtype=torch.float32)

  return [rope, visits]

In [148]:
get_initial_state()

[tensor([[500., 500.],
         [500., 500.],
         [500., 500.],
         [500., 500.],
         [500., 500.],
         [500., 500.],
         [500., 500.],
         [500., 500.],
         [500., 500.],
         [500., 500.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]])]

In [153]:
def move_head(head, direction):
  if direction == 'U':
    head[1] += 1
  elif direction == 'D':
    head[1] -= 1
  elif direction == 'L':
    head[0] -= 1
  elif direction == 'R':
    head[0] += 1
  else:
    raise Exception('Unknown direction')
  return head
  
def move_tail(head, tail):
  possible_moves = torch.tensor([[-1, 0], [1, 0], [0, -1], [0, 1], [1, 1], [-1, -1], [1, -1], [-1, 1]]) + tail
  distances = torch.cdist(head.unsqueeze(0), possible_moves)
  return possible_moves[torch.argmin(distances)]

def run_instruction(state, instruction):
  print(state)
  [rope, visits] = state
  direction = instruction['direction']
  distance = instruction['distance']
  for i in range(distance):
    rope[0] = move_head(rope[0], direction)
    for j in range(1, rope.shape[0]):
      head = rope[j-1]
      tail = rope[j]
      if (head.dist(tail) >= 2):
        tail = move_tail(head, tail)
      rope[j] = tail
    end = rope[-1]
    visits[int(end[0]), int(end[1])] += 1
    
  return [rope, visits]

In [154]:
[rope, visits] = run_instructions(parse_file(load_file('input.txt')))
(visits > 0).sum()

[tensor([[500., 500.],
        [500., 500.],
        [500., 500.],
        [500., 500.],
        [500., 500.],
        [500., 500.],
        [500., 500.],
        [500., 500.],
        [500., 500.],
        [500., 500.]]), tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])]
[tensor([[500., 501.],
        [500., 500.],
        [500., 500.],
        [500., 500.],
        [500., 500.],
        [500., 500.],
        [500., 500.],
        [500., 500.],
        [500., 500.],
        [500., 500.]]), tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])]
[tensor([[500., 499.],
        [500., 500.],

tensor(2653)